In [1]:
import os
import pandas as pd
import 

In [2]:
!ls data

dscovr	swpc


In [4]:
!find data -type f -name '*.csv'

data/swpc/kpindices-2022.csv
data/swpc/kpindices-2023.csv
data/dscovr/dsc_fc_summed_spectra_2022_v01.csv
data/dscovr/dsc_fc_summed_spectra_2023_v01.csv


In [5]:
dscovr_df = pd.read_csv("./data/dscovr/dsc_fc_summed_spectra_2022_v01.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
kp_df = pd.read_csv("./data/swpc/kpindices-2022.csv", delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)

In [6]:
dscovr_df.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2022-01-01 00:00:00,-6.17170,1.124830,-4.90228,38.0314,0.231726,46.0427,44.9743,55.9143,43.7069,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-01 00:01:00,-6.28883,1.233130,-4.79001,38.3868,0.231726,45.5257,46.2587,55.1428,43.2768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-01 00:02:00,-6.11811,0.871923,-5.12830,37.5636,0.231726,45.1955,46.8222,55.7484,42.7894,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-01 00:03:00,-6.28704,1.249870,-4.76640,38.1094,0.242084,46.7083,47.1713,53.5380,42.1558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-01 00:04:00,-6.42125,1.171560,-4.53230,37.5893,0.231726,47.4888,45.3234,54.5404,44.2773,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
kp_df.head()

,0,1,2,3,4,5,6,7,8
0,date,3.0,6.0,9.0,12.0,15.0,18.0,21.0,24.0
1,20220401,3.0,3.0,4.0,4.0,4.0,3.0,2.0,2.0
2,20220402,4.0,5.0,4.0,2.0,2.0,3.0,4.0,3.0
3,20220403,4.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0
4,20220404,4.0,4.0,3.0,2.0,2.0,2.0,2.0,1.0


We can see that the timestamp is different between data sources and we will need to fix that. Because we want to train on a relative measure of time we will convert the data into training examples with an integer ordinal number.

What other changes will be needed? 

- convert date and time to an ordinal count of seconds since the start time of one example
- convert the kp indices to an integer
- transform dscovr NaNs into 0 (for now - maybe there is a better value to use) 
- filter kp rows that have invalid data (-1s)
- normalize dscovr mag field values
- normalize dscovr solar wind values

The last transform we need to make is to organize the data from dscovr into training examples having a shape of (4320, 53, 2). In axis 0 we have 4,320 minutes per 3 days.
In axis 1 we have the 53 data points for each minute.
In axis 2 we have the ISO date timestamp of the Kp index we are learning and the Kp index we want to learn to predict.

Note that we will generate a new training example for every 3 hour Kp index for each day so we want to cache the pr with evious data and shift it for efficiency.

We need to save our training set to files with random training/valid/test splits. We will split 10% of the examples for validation and save them as 'valid.h5' and the remaining 90% will be our training examples stored as 'train.h5'.which is

We still don't have a held-out test set for evaluating the performance of our model. We will use the 2022 data for training and validation sets and the 2023 data as our test set. Of course, you could come back and change this by combining 2022 and 2023 into one big set then splitting it 3 ways, maybe with an 80/10/10 split which is commonly used.



In [ ]:
train_df.to_hdf('train.h5', key='train', mode='w', complevel=9)  
valid_df.to_hdf('valid.h5', key='valid', mode='w', complevel=9)
test_df.to_hdf('test.h5', key='valid', mode='w', complevel=9)